In [ ]:
import os
from pathlib import Path

import pandas as pd

# Tutorial for handling Modelica models 
The aim of this tutorial is to show how to generate boundary files (.txt) for Modelica models, to load Modelica models on python, set up and launch simulations using **Modelitool**.

# 1. Proposed model 

In this tutorial, we create of model of following wall, tested a "real scale" bench. The Nobatek BEF (Banc d'Essais Façade) provides experimental cells to test building façade solutions. The heat exchanges in a cell are limited on 5 of its faces. The 6th face is dedicated to the tested solution. Internal temperature and hydrometry conditions can be controlled or monitored. External conditions are measured (temperatures and solar radiation).  we propose a resistance/capacity approach.


| Figure : Pictures and model of the bench tested wall |
| :---: |
| <img src="images/etics_pict.png" style="height:200px;">  <img src="images/etics_sch.png"  style="height:200px;"> |

Based on electrical circuit analogy, each layer of the wall is modeled by two resistance and a capacity.
The model was built using openModelica : 

<img src="images/OM_eticsmodel.png"  height="200">

The following is a brief description of the thermal model, as it is not the scope of this document. See the <code>*mo</code> for full informations.

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - Resistances : $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $
    - Capacity : $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $

    
- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.


- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $
    - With the surrounding considered to be at $ T_{ext} $


- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.


- Temperatures $ T_{ext}$ and $T_{int} $ are boundary conditions


Initial conditions for the layers temperatures are taken from the measured data.

# 2. Set boundary file
## Option A: load csv file
Let's load measurement data on python. We can use this dataframe to define boundary conditions of our model.

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/study_df.csv", index_col=0, parse_dates=True
)

## Option B: Create boundary file for Modelica model
Or, before loading the Modelica model (*.mo), one might want to generate boundary files with the right format (.txt) to use it their model. For this, you can use combitabconvert from modelitool.

Make sure beforehand your data is clean: no NAs, non monotonically increasing index, abberant values, etc.

**_Note : Note that you have to manually configure the file path in
the <code>combiTimetable</code> of your modelica model_**

In [ ]:
from modelitool.combitabconvert import df_to_combitimetable

In [ ]:
df_to_combitimetable(
    df=reference_df.loc["2018-03-22":"2018-03-23"],
    filename="resources/boundary_temp.txt",
)

# 3. Load model from Modelica

To avoid loading all ouptut from modelica model, let's first define a list of output that will be included in the dataframe output for any simulation.

In [ ]:
output_list = ["T_coat_ins.T", "T_ins_ins.T", "Tw_out.T"]

Now, we can load the *om file.

The `OMModel` class is used to load and simulate Modelica models. It requires the following parameters:

- `model_path`: Path to the Modelica model file (*.mo) or model name if already loaded in OpenModelica
- `package_path` (optional): Path to additional Modelica packages required by the model
- `simulation_options` (optional): Dictionary containing simulation settings like:
  - `startTime`: Start time in seconds
  - `stopTime`: Stop time in seconds
  - `stepSize`: Time step for the simulation
  - `tolerance`: Numerical tolerance for the solver
  - `solver`: Solver to use (e.g. "dassl")
  - `outputFormat`: "mat" or "csv" for results format
  - `x`: Boundary conditions as a DataFrame (optional)
- `output_list` (optional): List of variables to include in simulation results
- `lmodel` (optional): List of required Modelica libraries (e.g. ["Modelica"])

In [ ]:
from modelitool.simulate import OMModel

In [ ]:
simu_OM = OMModel(
    model_path=Path(TUTORIAL_DIR) / "resources/etics_v0.mo",
    output_list=output_list,
    lmodel=["Modelica"],
)

#### Set up simulation options 

As they were not specified when instantiating OMModel, simulation running options (if different from the one provided by the modelica model) should be defined.

In Modelica, <code>startTime</code> and <code>stopTime</code> correspond to the number
of seconds since the beginning of the year. 

The values can be found in the file created earlier using <code>df_to_combitimetable</code> . Another way is to use the index of the <code>DataFrame</code> we just created.
The modelitool function <code>modelitool.combitabconvert.datetime_to_seconds</code>
helps you convert datetime index in seconds.


In [ ]:
from modelitool.combitabconvert import datetime_to_seconds

In [ ]:
simulation_df = reference_df.loc["2018-03-22":"2018-03-23"]
second_index = datetime_to_seconds(simulation_df.index)

- <code>stepSize</code> is the simulation timestep size. In this case it's 5 min or
300 sec.
- <code>tolerance</code> and <code>solver</code> are related to solver configuration
do not change if you don't need to.
- <code>outputFormat</code> can be either csv or mat. csv will enable faster data handling during sensitivity analyses and optimizations.
- <code>x</code>: as the boundary conditions. If not given here, it can still be provided in method `simulate`.

In [ ]:
simulation_opt = {
    "startTime": second_index[0],
    "stopTime": second_index[-1],
    "stepSize": 300,
    "tolerance": 1e-06,
    "solver": "dassl",
    "outputFormat": "csv",
}

# 4. Run the simulation

Set the initial and parameter values in a dictionary. They can either be set before simluation (with `set_param_dict()` method, or when using method `simulate()`. Each change of paramter value overwrite the previous one. 

In [ ]:
parameter_dict_OM = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
    "Lambda_ins.k": 0.04,
}

Simulation flags can also be specified in <code>simulate()</code> method. Overview of possible simulation flags can be found here: https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/simulationflags.html. Note that the simulation flag <code>override</code> cannot be used, as it was already used in class <code>OMModel</code> with <code>simulation_options</code>.

If x boundary conditions do not
    have a DateTime index (seconds int), a year can be specified to convert
    int seconds index to a datetime index. If simulation spans overs several
    years, it shall be the year when it begins.

The output of the `simulate()` method is a dataframe, containing the outputs listed in output_list.

In [ ]:
init_res_OM = simu_OM.simulate(
    simflags="-initialStepSize=60 -maxStepSize=3600 -w -lv=LOG_STATS",
    parameter_dict=parameter_dict_OM,
    x=reference_df,
    year=2024,
)
init_res_OM.head()

Plotted results

In [ ]:
init_res_OM.plot()